In [1]:
import sympy as sp
import numpy as np
import math

In [2]:
%timeit -n 100000 -r 10 sp.sqrt(100)

1.58 µs ± 17.1 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)


In [3]:
%timeit -n 100000 -r 10 np.sqrt(100)

966 ns ± 18.2 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)


In [4]:
%timeit -n 100000 -r 10 math.sqrt(100)

132 ns ± 8.03 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)


In [5]:
%timeit -n 100000 -r 10 100. ** 0.5

12.3 ns ± 4.86 ns per loop (mean ± std. dev. of 10 runs, 100000 loops each)
